In [ ]:
from transformers import BertConfig, BertModel

# Building the config
config = BertConfig()

# Building the model from the config
model = BertModel(config)


In [ ]:
print(config)

In [ ]:
print(model)

In [ ]:
from transformers import BertModel

checkpoint = "bert-base-cased"
model = BertModel.from_pretrained(checkpoint)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
model.classifier

### Model call

In [ ]:
sentenses = [
    "I’ve been waiting for a HuggingFace course my whole life.",
    "I hate this so much!"
]

# tokenizer.train_from_iterator(batched_sentences_generator)

token0 = tokenizer.tokenize(sentenses[0])
token1 = tokenizer.tokenize(sentenses[1])
tokens = [token0, token1]

In [ ]:
tokens

In [ ]:
ids = [tokenizer.convert_tokens_to_ids(t) for t in tokens]

In [ ]:
input0 = tokenizer(sentenses[0])
type(input0)

In [ ]:
input0.word_ids()

In [ ]:
inputs = tokenizer(sentenses, padding=True, return_tensors="pt")  # truncation=True, max_length=4)
inputs # = inputs.to(device)


In [ ]:
output = model(**inputs, output_hidden_states=True, output_attentions=True)

In [ ]:
output.keys()

In [ ]:
print(model.config.id2label)
print(model.config.label2id)

output.logits

In [ ]:
type(output.hidden_states), len(output.hidden_states), output.hidden_states[0].shape

In [ ]:
type(output.attentions), len(output.attentions), output.attentions[0].shape

In [ ]:
import torch

inputs['labels'] = torch.tensor([0, 1], dtype=torch.int64)

In [ ]:
output = model(**inputs)
output.keys()

In [ ]:
model.eval()
with torch.inference_mode():
    print(torch.nn.functional.softmax(model(**inputs).logits, dim=-1))

### Datasets

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
# mine_datasets = load_dataset("text", data_files={'train': "data/mrpc.txt", 'test': '...'})

In [ ]:
raw_datasets

In [ ]:
raw_train_dataset = raw_datasets['train']

In [ ]:
raw_train_dataset[0], len(raw_train_dataset)

In [ ]:
raw_train_dataset.features

#### Preprocessing

In [ ]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
def tokenize_function(example):
    # dict[str, list | np.array | torch.tesnor]
    # returns dict[str, list | np.array | torch.tesnor]
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenize_function({'sentence1': 'abc', 'sentence2': 'cde'})

In [ ]:
tokenized_datasets_1 = raw_datasets.map(tokenize_function, num_proc=2, remove_columns=['sentence1', 'sentence2'])
tokenized_datasets_1

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True, num_proc=2, remove_columns=['sentence1', 'sentence2'])
tokenized_datasets

In [ ]:
raw_train_dataset[0]

In [ ]:
raw_train_dataset[0:2]

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True, num_proc=2, remove_columns=['sentence1', 'sentence2'])
tokenized_datasets

In [ ]:
tokenized_datasets.save_to_disk('data/glue/mrpc/tokenized')

from datasets import load_from_disk
tokenized_datasets = load_from_disk('data/glue/mrpc/tokenized')

In [ ]:
tokenized_datasets

In [ ]:
from datasets import Dataset
from pandas import DataFrame

dataset = Dataset.from_pandas(DataFrame({'a' : [1, 2, 3]}))
dataset

#### dynamic batching

In [ ]:
samples = tokenized_datasets["train"][:8]
samples = {
    k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]
}
[len(x) for x in samples["input_ids"]]


In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

### Trainer

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments("test-trainer")

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

#### trainer with metrics

In [ ]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch", num_train_epochs=5)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer_output = trainer.train()

#### complete example for SST-2

In [ ]:
from datasets import load_dataset, load_metric
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding, 
    TrainingArguments,
    Trainer,
)

In [ ]:
raw_datasets = load_dataset("glue", "sst2")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import numpy as np

def compute_metrics(eval_preds):
    metric = load_metric("glue", "sst2")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
training_args = TrainingArguments(
    "test-trainer", 
    evaluation_strategy="epoch", # save_strategy, log_strategy, 
    num_train_epochs=3, 
    per_device_train_batch_size=16,
    report_to=None,
    # a lot of other arguments
)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)